In [1]:
import os
import re
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.common import exceptions
import time

In [2]:
chrome_driver = "../driver/chromedriver.exe"
link = "https://steamcommunity.com/app/567640/discussions/3/1520386297703978107/"

In [5]:
next_page_button_id = "commentthread_ForumTopic_103582791459514491_1458455461496305158_1520386297703978107_pagebtn_next"

In [6]:
%%time
driver = webdriver.Chrome(chrome_driver)
driver.get(link)

pages = {}
page_num = 0
pages_remaining = True
while pages_remaining:

    # Wait 5-15 secs
    time.sleep(random.randrange(5,15))

    # Save sources
    pages[page_num] = driver.page_source
    page_num += 1
    print(page_num,"pages saved.")

    # Next page
    driver.find_element_by_id(next_page_button_id).click()
    
    # Check if next page exists. 
    # This button is always visible and clickable; can only use href value to determine last page
    pages_remaining = (re.search('javascript:void\(0\);', driver.find_element_by_id(next_page_button_id).get_attribute("href")) is None)

1 pages saved.
2 pages saved.
3 pages saved.
4 pages saved.
5 pages saved.
6 pages saved.
7 pages saved.
8 pages saved.
9 pages saved.
10 pages saved.
11 pages saved.
Wall time: 1min 47s


In [7]:
# Parse out comment ID and contents
all_ids = []
all_comments = []
for k in pages:
    soup = BeautifulSoup(pages[k],'html.parser')
    
    page_k_ids = soup.findAll("div",{"class":"forum_comment_permlink"})
    page_k_ids = [re.sub("#","", id_block.text.strip()) for id_block in page_k_ids]
    all_ids = all_ids + page_k_ids
    
    page_k_comments = soup.findAll("div",{"class":"commentthread_comment_text"})
    all_comments = all_comments + page_k_comments

comments_dict = dict(zip(all_ids, all_comments))

In [8]:
# Handle nested quotes
comments_no_quote_dict = {}
comments_quotes = {}  # keep track of it. Might wanna do something later.

for k in comments_dict:
    # .find() will get the first quoteblock that nests all the previous qoutes inside
    if comments_dict[k].find("blockquote") is None:
        comments_no_quote_dict[int(k)] = comments_dict[k].text.strip()
    else:
        comments_quotes[int(k)] = comments_dict[k].find("blockquote")
        comments_no_quote_dict[int(k)] = comments_dict[k].find("blockquote").replaceWith('').text.strip()

In [9]:
comments_df = pd.DataFrame.from_dict(comments_no_quote_dict,orient="index").sort_index()
comments_df[:3]

,0
1,Miu is like a walking personality complex and ...
2,Maki Harukawa. Nobody can convince me otherwise!
3,"Not only my favorite of this game, but now bet..."


In [10]:
quotes_df = pd.DataFrame.from_dict(comments_quotes,orient="index").sort_index()
quotes_df[:3]

,0
9,"<blockquote class=""bb_blockquote with_author"">..."
14,"<blockquote class=""bb_blockquote with_author"">..."
27,"<blockquote class=""bb_blockquote with_author"">..."


In [11]:
# check if it's safe to use pie as delimiter
print(sum(quotes_df[0].map(lambda x: re.search("|",str(x)) is None)))
print(sum(comments_df[0].map(lambda x: re.search("|",str(x)) is None)))

0
0


In [12]:
# Write the clean comments to file
comments_df.to_csv("../data/clean_comments.txt",sep="|")
quotes_df.to_csv("../data/quotes.txt",sep="|")